In [1]:
import pandas as pd
import MySQLdb
import sqlalchemy

ModuleNotFoundError: No module named 'MySQLdb'

In [14]:
df = pd.read_csv('mysql.csv', sep = ';')

In [15]:
df.head()

,jogador,genero,data_nascimento,jogo,pais,num_vitorias,num_derrotas,total_partidas
0,jogador 1,Masculino,03/09/1990,Dama,Chile,14.0,0.0,14.0
1,Jogador 2,Feminino,01/05/1986,Jogo da velha,Chile,39.0,20.0,59.0
2,Jogador 3,Masculino,04/10/1964,Jogo da velha,Brasil,26.0,3.0,29.0
3,Jogador 4,Masculino,28/09/1979,Jogo da velha,Argentina,9.0,72.0,81.0
4,Jogador 5,Masculino,03/09/1978,Dama,Uruguai,1.0,54.0,55.0


In [16]:
df.describe()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 499 entries, 0 to 498
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   jogador          499 non-null    object 
 1   genero           499 non-null    object 
 2   data_nascimento  499 non-null    object 
 3   jogo             499 non-null    object 
 4   pais             499 non-null    object 
 5   num_vitorias     491 non-null    float64
 6   num_derrotas     494 non-null    float64
 7   total_partidas   491 non-null    float64
dtypes: float64(3), object(5)
memory usage: 31.3+ KB


In [17]:
df.isna().sum()

jogador            0
genero             0
data_nascimento    0
jogo               0
pais               0
num_vitorias       8
num_derrotas       5
total_partidas     8
dtype: int64

In [18]:
df.num_vitorias.fillna(df['total_partidas'] - df['num_derrotas'], inplace = True)
df.num_derrotas.fillna(df['total_partidas'] - df['num_vitorias'], inplace = True)
df.total_partidas.fillna(df['num_vitorias'] + df['num_derrotas'], inplace = True)
df.isna().sum()

jogador            0
genero             0
data_nascimento    0
jogo               0
pais               0
num_vitorias       0
num_derrotas       0
total_partidas     0
dtype: int64

In [19]:
lst_pais = list(df.pais.unique())
lst_jogo = list(df.jogo.unique())

In [20]:
usr = 'teste'
pwd = 'teste'
hst = '127.0.0.1'
dbs = 'teste'
str = f'mysql://{usr}:{pwd}@{hst}/{dbs}'

con = sqlalchemy.create_engine(str).connect()

In [21]:
def sql(query):
  try:
    con.execute(query)
  except Exception as e:
    print(f'Erro: {e} | Query: "{query}"')

In [22]:
sql(f'delete from jogador;')
sql(f'delete from pais;')
sql(f'delete from jogo;')

In [23]:
dic_pais = {}
for cod, pais in enumerate(lst_pais):
  query = f'insert into pais values({cod + 1}, \'{pais}\');'
  sql(query)
  dic_pais[pais] = cod + 1

In [24]:
dic_jogo = {}
for cod, jogo in enumerate(lst_jogo):
  query = f'insert into jogo values({cod + 1}, \'{jogo}\');'
  sql(query)
  dic_jogo[jogo] = cod + 1

In [25]:
# df.data_nascimento
# pd.to_datetime(df.data_nascimento)
# df['data_nascimento1'] = pd.to_datetime(df.data_nascimento)
df['data_nascimento'] = pd.to_datetime(df.data_nascimento)
df.pais = df.pais.map(dic_pais)
df.jogo = df.jogo.map(dic_jogo)
df

,jogador,genero,data_nascimento,jogo,pais,num_vitorias,num_derrotas,total_partidas
0,jogador 1,Masculino,1990-03-09,1,1,14.0,0.0,14.0
1,Jogador 2,Feminino,1986-01-05,2,1,39.0,20.0,59.0
2,Jogador 3,Masculino,1964-04-10,2,2,26.0,3.0,29.0
3,Jogador 4,Masculino,1979-09-28,2,3,9.0,72.0,81.0
4,Jogador 5,Masculino,1978-03-09,1,4,1.0,54.0,55.0
...,...,...,...,...,...,...,...,...
494,Jogador 495,Feminino,1974-03-06,1,1,52.0,28.0,80.0
495,Jogador 496,Masculino,1959-09-02,1,1,13.0,51.0,64.0
496,Jogador 497,Masculino,1962-06-01,2,3,57.0,23.0,80.0
497,Jogador 498,Masculino,1958-09-28,2,1,5.0,3.0,8.0


In [26]:
for j in df.itertuples():
  query = f'insert into jogador values({j.Index + 1}, \'{j.jogador}\', \'{j.genero}\', \'{j.data_nascimento}\', {j.num_vitorias}, {j.num_derrotas}, {j.total_partidas}, {j.pais}, {j.jogo});'
  # print(query)
  sql(query)